# Hull Tactical Market Prediction - Fixed Submission

**FIXED**: Returns signal (0.0-2.0) instead of raw prediction

In [1]:
import os
import sys
from pathlib import Path

import joblib
import numpy as np
import pandas as pd
import polars as pl

import kaggle_evaluation.default_inference_server

print(f'Python: {sys.version}')
import sklearn
print(f'sklearn: {sklearn.__version__}')

Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
sklearn: 1.2.2


In [2]:
# Load dataset and models
dataset_path = Path('/kaggle/input/htmp-baseline-models')
sys.path.insert(0, str(dataset_path))

from src.features import FeatureConfig, SimpleFeatureExtractor

print(f'Dataset: {dataset_path}')
print(f'Contents: {os.listdir(dataset_path)}')

Dataset: /kaggle/input/htmp-baseline-models
Contents: ['models', 'src']


In [3]:
# Signal conversion parameters (from official starter notebook)
SIGNAL_MULTIPLIER = 400.0
MIN_SIGNAL = 0.0
MAX_SIGNAL = 2.0

def convert_ret_to_signal(ret_pred: float) -> float:
    """Convert raw return prediction to trading signal (0.0-2.0).
    
    0.0 = Short position (market down)
    1.0 = Neutral (no position)
    2.0 = Long position (market up)
    """
    signal = ret_pred * SIGNAL_MULTIPLIER + 1.0
    return float(np.clip(signal, MIN_SIGNAL, MAX_SIGNAL))

print('Signal conversion function defined')
print(f'Example: ret=0.006 -> signal={convert_ret_to_signal(0.006):.3f}')
print(f'Example: ret=0.0 -> signal={convert_ret_to_signal(0.0):.3f}')
print(f'Example: ret=-0.003 -> signal={convert_ret_to_signal(-0.003):.3f}')

Signal conversion function defined
Example: ret=0.006 -> signal=2.000
Example: ret=0.0 -> signal=1.000
Example: ret=-0.003 -> signal=0.000


In [4]:
# Load models
models = []
extractors = []
feature_cols = None

DROP_COLUMNS = [
    'date_id', 'forward_returns', 'risk_free_rate', 'market_forward_excess_returns',
    'lagged_forward_returns', 'lagged_risk_free_rate', 'lagged_market_forward_excess_returns', 'is_scored'
]

model_files = sorted((dataset_path / 'models').glob('lgbm_augmented_drop40_fold_*.pkl'))
print(f'Found {len(model_files)} model files')

for mf in model_files:
    artifact = joblib.load(mf)

    model = artifact["model"]
    ext = artifact["feature_extractor"]

    # ✅ 安全確認（ここで落ちるなら学習が壊れている）
    assert hasattr(ext, "filtered_columns") and len(ext.filtered_columns) > 0, \
        "FeatureExtractor is not fitted at save time"

    models.append(model)
    extractors.append(ext)
    
    # config = FeatureConfig(
    #     drop_columns=DROP_COLUMNS,
    #     imputation_strategy='median',
    #     scale=True,
    #     rolling_windows=None,
    #     enable_interactions=True,
    #     time_column='date_id',
    #     group_column=None,
    #     missing_ratio_threshold=40.0
    # )
    # ext = SimpleFeatureExtractor(config)
    # ext.scaler = artifact.get('scaler')
    # ext.numeric_columns = artifact['feature_columns']
    # extractors.append(ext)
    
    # if feature_cols is None:
    #     feature_cols = artifact['feature_extractor'].selected_columns_
    
    print(f'✅ Loaded {mf.name}')

print(f'\n✅ All models loaded: {len(models)} folds')
print(f'✅ Feature columns: {len(ext.filtered_columns)}')

Found 5 model files
✅ Loaded lgbm_augmented_drop40_fold_0.pkl
✅ Loaded lgbm_augmented_drop40_fold_1.pkl
✅ Loaded lgbm_augmented_drop40_fold_2.pkl
✅ Loaded lgbm_augmented_drop40_fold_3.pkl
✅ Loaded lgbm_augmented_drop40_fold_4.pkl

✅ All models loaded: 5 folds
✅ Feature columns: 86


In [5]:
# Define predict function - RETURNS SIGNAL (0.0-2.0) not raw prediction
def predict(test: pl.DataFrame) -> float:
    """Predict trading signal (0.0-2.0).
    
    Args:
        test: Polars DataFrame with test features
        
    Returns:
        float: Trading signal between 0.0 (short) and 2.0 (long)
    """
    try:
        # Convert to pandas
        test_df = test.to_pandas()
        
        if len(test_df) == 0:
            return 1.0  # Neutral signal for empty input
        
        if len(test_df) > 1:
            test_df = test_df.iloc[[0]]
        
        # Make predictions with each fold
        fold_preds = []
        for model, ext in zip(models, extractors):
            transformed = ext.transform(test_df, target_column='market_forward_excess_returns')
            
            # Align columns
            # for col in feature_cols:
            #     if col not in transformed.columns:
            #         transformed[col] = 0.0
            # transformed = transformed[feature_cols]
            
            # Predict
            pred = model.predict(transformed)
            fold_preds.append(pred[0])
        
        # Average across folds
        raw_pred = float(np.mean(fold_preds))
        
        # Convert to signal (0.0-2.0) - THIS IS THE KEY FIX!
        signal = convert_ret_to_signal(raw_pred)
        
        return signal
        
    except Exception as e:
        print(f'ERROR in predict: {e}')
        import traceback
        traceback.print_exc()
        return 1.0  # Neutral signal on error

print('✅ Predict function defined')

✅ Predict function defined


In [6]:
# Initialize and start server
inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    print('🚀 Starting inference server for competition submission...')
    inference_server.serve()
else:
    print('🧪 Running local gateway for testing...')
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))

🧪 Running local gateway for testing...


In [7]:
print(len(ext.base_columns))
print(len(ext.filtered_columns))
print(len(ext.impute_values))
print(type(ext.scaler))

94
86
86
<class 'sklearn.preprocessing._data.StandardScaler'>


In [8]:
train_df = pd.read_csv("/kaggle/input/hull-tactical-market-prediction/train.csv")
X_debug = ext.transform(train_df.head(1), target_column="market_forward_excess_returns")
print(X_debug.shape)

(1, 1162)
